# TODO GO to KRM and create a volume first with name "volume-sentinel"

In krm create first a volume otherwise the RUN is stuck waiting for the volume
persisten_volume_claim specs:
- volume_name : volume-sentinel # this name is to be equal to the "name parameter inside volumes during the run of the function"
- space : 15Gbi
- storage_class_name: #disk
- access_mode: readWriteOnce

If you have any doubts on the params check the README.md inside https://github.com/tn-aixpa/sentinel-tools 

In [ ]:
import digitalhub as dh
# conda install -c conda-forge gdal
PROJECT_NAME = "docker-sentinel"
proj = dh.get_or_create_project(PROJECT_NAME) # source="git://github.com/scc-digitalhu

Register to the open data space copenicus(if not already) and get your credentials.

https://identity.dataspace.copernicus.eu/auth/realms/CDSE/login-actions/registration?client_id=cdse-public&tab_id=FIiRPJeoiX4

Log the credentials as project secret keys as shown below

In [2]:
# THIS NEED TO BE EXECUTED JUST ONCE
secret0 = proj.new_secret(name="CDSETOOL_ESA_USER", secret_value="esa_username")
secret1 = proj.new_secret(name="CDSETOOL_ESA_PASSWORD", secret_value="esa_password")

### Sentinel 1

In [8]:
string_dict_data = """{
  'satelliteParams':{
      'satelliteType': 'Sentinel1',
      'processingLevel': 'LEVEL1',
      'sensorMode': 'IW',
      'productType': 'SLC'
  } ,
  'startDate': '2023-12-12',
  'endDate': '2023-12-13',
  'geometry': 'POLYGON((10.98014831542969 45.455314263477874,11.030273437500002 45.44808893044964,10.99937438964844 45.42014226680115,10.953025817871096 45.435803739956725,10.98014831542969 45.455314263477874))',
  'area_sampling': 'True',
  'tmp_path_same_folder_dwl':'True',
  'artifact_name': 'sentinel1_all'
  }"""

# s3 path is not mandatory

list_args =  ["main.py",string_dict_data]
function_s1 = proj.new_function("download_images_s1",kind="container",image="ghcr.io/tn-aixpa/sentinel-tools:0.10.0",command="python")

In [9]:
run = function_s1.run(action="job",
        secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
        fsGroup='8877',
        args=list_args,           
        volumes=[{
            "volume_type": "persistent_volume_claim",            
            "name": "volume-sentinel", # this name has to be equal to the name of the volume created in krm
            "mount_path": "/files",
            "spec": {
                "claim_name": "volume-sentinel"
    }}],)

### Sentinel 2

In [10]:
string_dict_data = """{
 "satelliteParams":{
     "satelliteType": "Sentinel2",
	 "bandmath": ["NDWI"]	 
 },
 "startDate": "2020-09-15",
 "endDate": "2020-10-15",
 "geometry": "POLYGON((10.98014831542969 45.455314263477874,11.030273437500002 45.44808893044964,10.99937438964844 45.42014226680115,10.953025817871096 45.435803739956725,10.98014831542969 45.455314263477874))",
 "area_sampling": "true",
 "cloudCover": "[0,2]",
 "artifact_name": "sentinel2-flood-2020" 
 }"""

list_args =  ["main.py",string_dict_data]
function_s2 = proj.new_function("download_images_s2",kind="container",image="ghcr.io/tn-aixpa/sentinel-tools:0.10.0",command="python")

In [11]:
run = function_s2.run(action="job",
        secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
        fsGroup='8877',
        args=list_args,           
        volumes=[{
            "volume_type": "persistent_volume_claim",            
            "name": "volume-sentinel", # this name has to be equal to the name of the volume created in krm
            "mount_path": "/files",
            "spec": {
                "claim_name": "volume-sentinel"
    }}],)

#### Processing level

In [ ]:
string_dict_data = """{
 "satelliteParams":{
    "satelliteType": "Sentinel2",
    "processingLevel": "S2MSI2A",
	"bandmath": ["NDVI","EVI","NDWI","NDSI","NBR"]	 
 },
 "startDate": "2023-12-12",
 "endDate": "2023-12-30",
 "geometry": "POLYGON((10.88558452267069 46.2069331490752, 11.02591468396198 46.2069331490752, 11.02591468396198 46.288250617785245, 10.88558452267069 46.288250617785245, 10.88558452267069 46.2069331490752))",
 "cloudCover": "[0,5]",
 "area_sampling": "True",
 "artifact_name": "sentinel2_preprocess_only",
 "preprocess_data_only": "true"
 }"""

list_args =  ["main.py",string_dict_data]
function_s2_pl = proj.new_function("download_images_s2_pl",kind="container",image="ghcr.io/tn-aixpa/sentinel-tools:0.10.0",command="python")

In [ ]:
run = function_s2_pl.run(action="job",
        secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
        fsGroup='8877',
        args=list_args,           
        volumes=[{
            "volume_type": "persistent_volume_claim",            
            "name": "volume-sentinel", # this name has to be equal to the name of the volume created in krm
            "mount_path": "/files",
            "spec": {
                "claim_name": "volume-sentinel"
    }}],)